In [1]:
# Importing Required modules
import pandas as pd
import mysql.connector as msql
import math
import os
import datetime
import re

In [2]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
# Specifying the directory path where the Excel files are located
directory_path = r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\data_files\01 General Info"


# Creating a list of Excel files in the specified directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Looping through each Excel file
for file in excel_files:
    # Constructing the full file path
    file_path = os.path.join(directory_path, file)
    
    # Reading the Excel file into a DataFrame
    gi = pd.read_excel(file_path)
    
    # Printing the name of the file
    print(f"Data from {file}:")

Data from General Information.xlsx:


In [4]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
gi['Email ID'] = gi['Email ID'].apply(clean_email)

In [5]:
#Reading She for STEM Incubator file present on source files which came form gd-graphy data
directory_path =(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\data_files\02 Graphy Data")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    gd = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from Kalpana Accelerator May - Sept 2024 Batch-1731667710527.csv:


In [6]:
# Define a list of column names to extract for putting into database 
columns_to_extract = ['Name', 'Email', 'Segment', 'Mobile', 'Enroll Date', 'Assigned Through', 'Course Name','Last Login','Start Date']

# Use the loc method to extract the specified columns
ce = gd.loc[:, columns_to_extract]

In [8]:
# Create a new dataframe with 'email' column and columns that start with 'Week' 'Video', 'Recording', and 'Master class'
gd = gd[['Email'] + [col for col in gd.columns if col.startswith(('Week','Video', 'SUK', 'Master Class', 'MM'))]]


In [9]:
# Create a datalist with gd dataframe
oldcol = list(gd.columns)

In [10]:
# Initialize variables to keep track of the current week, video count, recording count, and master class count
week_col = None
video_count = 0
recording_count = 0
master_count = 0
mm_count = 0

# Create an empty list to store the new column names
new_cols = []

# Iterate over each column in the gd
for col in gd.columns:
    # If the column starts with 'Week'
    if col.startswith('Week'):
        # Split the column name by space and get the second element (the week number)
        week_col = col.split()[1]
        # Reset the video, recording, and master class counts for the new week
        video_count = 0
        recording_count = 0
    # If the column starts with 'Video'
    elif col.startswith('Video'):
        # Increment the video count for the current week
        video_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_V{video_count}')
    # If the column starts with 'Recording'
    elif col.startswith('SUK'):
        # Increment the recording count for the current week
        recording_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_SUK_V')
    # If the column starts with 'Master Class'
    elif col.startswith('Master Class'):
        # Increment the master class count for the current week
        master_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_Master{master_count}')
    elif col.startswith('MM'):
        # Increment the MM count for the current week
        mm_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_MM{mm_count}')
    # If the column doesn't start with any of the above prefixes
    else:
        # Append the original column name to the list
        new_cols.append(col)

# Remove all columns that start with 'Week' from the gd
gd = gd.loc[:, ~gd.columns.str.startswith('Week')]
# Assign the new column names to the gd
gd.columns = new_cols


In [11]:
# Cheaking columns name
gd.columns

Index(['Email', 'WKNone_SUK_V', 'WKNone_MM1', 'WK1_Master1', 'WK2_V1',
       'WK2_V2', 'WK2_V3', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4',
       'WK3_Master2', 'WK4_V1', 'WK4_V2', 'WK4_MM2', 'WK5:_V1', 'WK5:_V2',
       'WK5:_V3', 'WK5:_Master3', 'WK6_SUK_V', 'WK7_V1', 'WK7_V2', 'WK7_V3',
       'WK7_Master4', 'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V',
       'WK9_V1', 'WK9_V2', 'WK9_Master5', 'WK10_SUK_V'],
      dtype='object')

In [12]:
# Rename columns starting with 'WKNone' to 'WK0' and removing : if present
gd.columns = [
    col.replace('WKNone', 'WK0').replace(':', '') if col.startswith('WKNone') or ':' in col else col
    for col in gd.columns]

In [13]:
# Locating columns for extracting only numbers
col1 = gd.iloc[:,1:]

In [14]:
# Code for extracting only numbers from dataset
for column in [i for i in col1.columns if col1[i].dtype == 'object']:
    gd[column] = gd[column].astype(str).str.extract('(\d+)').astype(float)

In [15]:
# Change all columns to object type
#col1 = col1.astype(object)

# Merging data from Live Session Attendance Sheet

In [16]:
# Reading Excel file form our source
df1  = pd.read_excel(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\data_files\03 Live Session Attendance\Live Session Attendance.xlsx")

In [17]:
# Renameing the column name if it is in dataframe
if 'Enter the email ID that you use to login to the VigyanShaala app/platform' in df1.columns: df1.rename(columns={'Enter the email ID that you use to login to the VigyanShaala app/platform': 'Email ID'}, inplace=True)

In [18]:
# Function to clean email addresses

def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
df1['Email ID'] = df1['Email ID'].apply(clean_email)

In [19]:
df1 = pd.merge(gi, df1, on='Email ID', how='outer')

In [20]:
# Get the columns to keep
columns_to_keep = [col for col in df1.columns if col.startswith("Email ID") or col.startswith("Start time")]

# Drop the columns not in columns_to_keep
df1.drop(columns=[col for col in df1.columns if col not in columns_to_keep], inplace=True)


In [21]:
# Remove rows where 'Start time' is null
df1 = df1.dropna(subset=['Start time'])

In [22]:
if 'Start time' in df1.columns: df1.rename(columns={'Start time': 'Timestamp'}, inplace=True)

In [ ]:
# URL of the Google Sheets document

url = ''
# The URL needs to be modified to point to the export format

url = url.replace('/edit?usp=sharing', '/export?format=xlsx')

# Read the data from the URL into a pandas DataFrame

df2 = pd.read_excel(url)


In [24]:
# Get the columns to keep
columns_to_keep = [col for col in df2.columns if col.startswith("Timestamp") or col.startswith("Select your ID & Name") ]

# Drop the columns not in columns_to_keep
df2.drop(columns=[col for col in df2.columns if col not in columns_to_keep], inplace=True)


In [26]:
# Convert the column to string, handling NaNs by converting them to empty strings or some other placeholder
df2['Select your ID & Name'] = df2['Select your ID & Name'].astype(str)

In [27]:
# Extract the required part
df2['Mentee ID'] = df2['Select your ID & Name'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])




In [28]:
# Outer merge of two dataframe to make it one
df2 = pd.merge(gi, df2, on='Mentee ID', how='outer')

In [29]:
# Get the columns to keep
columns_to_keep = [col for col in df2.columns if col.startswith("Email ID") or col.startswith("Timestamp") ]

# Drop the columns not in columns_to_keep
df2.drop(columns=[col for col in df2.columns if col not in columns_to_keep], inplace=True)


In [30]:
# Concating two dataframe to make it one
la = pd.concat([df1, df2], ignore_index=True)

In [31]:
# Checking shape of dataframe
la.shape

(780, 2)

In [32]:
# Remove rows where 'Start time' is null
la = la.dropna(subset=['Timestamp'])

In [33]:
# Remove rows where 'Email ID' is null
la = la.dropna(subset=['Email ID'])

In [34]:
# Convert Timestamp column to datetime
la['Timestamp'] = pd.to_datetime(la['Timestamp'])

# Create a new column 'dates' with the desired format
la['Date'] = la['Timestamp'].dt.strftime('%Y-%m-%d')

In [35]:
# Drop the Timestamp column
la = la.drop(columns=['Timestamp'])

###  👇 The date and the Name of session should be same so it will get merge 👇

In [36]:
# Reading Excel file form our source
ld  = pd.read_excel(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\data_files\03 Live Session Attendance\Live Session Dates.xlsx")

In [37]:
# Convert 'Date' column to datetime format
la['Date'] = pd.to_datetime(la['Date'])
ld['Date'] = pd.to_datetime(ld['Date'])

# Merge dataframes on 'Date'
md = pd.merge(la, ld, on='Date', how='outer')


In [38]:
# Remove the 'name' column
md = md.drop(columns=['Date'])

# Add a new column 'duration' with all values set to 3600
md['Duration'] = 3600

In [39]:
# Create a pivot table for Session
md_pivot = md.pivot_table(index='Email ID', columns='Sessions')

In [40]:
# Creating new column names by combining the first and second elements
md_pivot.columns = [f'{col[0]} - {col[1]}' for col in md_pivot.columns]


In [41]:
# Reseting the dataframe
md_pivot.reset_index(inplace=True)

In [42]:
# Renaming columns of dataframe
md_pivot.columns = md_pivot.columns.str.replace(r'Duration - ', '')


In [43]:
# Reaname the dataframe if we find that column name
if 'Email ID' in md_pivot.columns: md_pivot.rename(columns={'Email ID': 'Email'}, inplace=True)

In [45]:
# Merge dataframes on 'Email' column
merged_df = pd.merge(gd, md_pivot, on='Email', how='outer', suffixes=('_df', '_da'))

# List of columns to sum
columns_to_sum = [col for col in md_pivot.columns if col != 'Email']

# Sum the matching columns
for col in columns_to_sum:
    merged_df[col] = merged_df[f'{col}_df'].fillna(0) + merged_df[f'{col}_da'].fillna(0)

# Drop the intermediate columns
merged_df.drop(columns=[f'{col}_df' for col in columns_to_sum] + [f'{col}_da' for col in columns_to_sum], inplace=True)



In [46]:
# Checking the rows of gd dataframe
filtered_df = gd[gd['Email'] == 'ksharry799@gmail.com']
print(filtered_df.head())


                   Email  WK0_SUK_V  WK0_MM1  WK1_Master1  WK2_V1  WK2_V2  \
59  ksharry799@gmail.com        NaN    152.0          NaN     NaN     NaN   

    WK2_V3  WK2_SUK_V  WK3_V1  WK3_V2  WK3_V3  WK3_V4  WK3_Master2  WK4_V1  \
59     NaN        NaN     NaN     NaN     NaN     NaN          NaN     NaN   

    WK4_V2  WK4_MM2  WK5_V1  WK5_V2  WK5_V3  WK5_Master3  WK6_SUK_V  WK7_V1  \
59     NaN      NaN     NaN     NaN     NaN          NaN        NaN     NaN   

    WK7_V2  WK7_V3  WK7_Master4  WK8_V1  WK8_V2  WK8_V3  WK8_V4  WK8_SUK_V  \
59     NaN     NaN          NaN     NaN     NaN     NaN     NaN        NaN   

    WK9_V1  WK9_V2  WK9_Master5  WK10_SUK_V  
59     NaN     NaN          NaN         NaN  


In [47]:
# checking if the merge is happend or not
filtered_df_1 = merged_df[merged_df['Email'] == 'ksharry799@gmail.com']
print(filtered_df_1.head())


                   Email  WK0_MM1  WK2_V1  WK2_V2  WK2_V3  WK3_V1  WK3_V2  \
59  ksharry799@gmail.com    152.0     NaN     NaN     NaN     NaN     NaN   

    WK3_V3  WK3_V4  WK4_V1  WK4_V2  WK5_V1  WK5_V2  WK5_V3  WK7_V1  WK7_V2  \
59     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

    WK7_V3  WK8_V1  WK8_V2  WK8_V3  WK8_V4  WK9_V1  WK9_V2  WK0_SUK_V  \
59     NaN     NaN     NaN     NaN     NaN     NaN     NaN     3600.0   

    WK10_SUK_V  WK1_Master1  WK2_SUK_V  WK3_Master2  WK4_MM2  WK5_Master3  \
59         0.0          0.0        0.0          0.0      0.0          0.0   

    WK6_SUK_V  WK7_Master4  WK8_SUK_V  WK9_Master5  
59        0.0          0.0        0.0          0.0  


In [48]:
# Ensure that merged_df contains all columns from gd
missing_cols = [col for col in gd.columns if col not in merged_df.columns]
for col in missing_cols:
    merged_df[col] = pd.NA

# Reorder the columns of merged_df to match the order of gd.columns
merged_df = merged_df[gd.columns]

In [49]:
# Checking the columns name
merged_df.columns

Index(['Email', 'WK0_SUK_V', 'WK0_MM1', 'WK1_Master1', 'WK2_V1', 'WK2_V2',
       'WK2_V3', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4',
       'WK3_Master2', 'WK4_V1', 'WK4_V2', 'WK4_MM2', 'WK5_V1', 'WK5_V2',
       'WK5_V3', 'WK5_Master3', 'WK6_SUK_V', 'WK7_V1', 'WK7_V2', 'WK7_V3',
       'WK7_Master4', 'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V',
       'WK9_V1', 'WK9_V2', 'WK9_Master5', 'WK10_SUK_V'],
      dtype='object')

# Payment details MySQL table

In [50]:
# Creating new table which for Payment which is taken from incubator graphy sheet
Payment=pd.DataFrame(ce[["Email","Assigned Through"]])

In [51]:
# Changing column name to name present in database
Payment = Payment.rename(columns={'Assigned Through': 'Accelerator_Fee'})

In [52]:
# Assigning fee cost to different cathegory of enrollment
Payment["Accelerator_Fee"]=Payment["Accelerator_Fee"].replace(['Admin'],'0')
Payment["Accelerator_Fee"]=Payment["Accelerator_Fee"].replace(['Excel Upload'],'0')
Payment["Accelerator_Fee"]=Payment["Accelerator_Fee"].replace(['Paid Transaction '],'1899')

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

In [54]:
# Inserting data into 02_payment_details
for i, row in Payment.iterrows():
    cursor.execute(
        "INSERT IGNORE INTO 02_payment_details (Email, Accelerator_Fee) VALUES (%s, %s)",
        (row['Email'], row['Accelerator_Fee'])
    )

In [55]:
# Replacing data with new data into 02_payment_details
for i, row in Payment.iterrows():
    cursor.execute(
        "REPLACE INTO 02_payment_details (Email, Accelerator_Fee) VALUES (%s, %s)",
        (row['Email'], row['Accelerator_Fee'])
    )

In [56]:
conn.commit()

# Enroll Date MySQL Table

In [57]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Enroll=pd.DataFrame(ce[["Email", 'Enroll Date']])

In [58]:
# Extracting only Enroll_Date
Enroll[['Enroll Date','Time']]=Enroll['Enroll Date'].str.split(' ',expand=True)
Enroll=Enroll.drop(["Time"],axis=1)

In [59]:
# Changing column name to name present in database
Enroll = Enroll.rename(columns={'Enroll Date': 'Accelerator'})

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

In [61]:
print(Enroll.columns)

Index(['Email', 'Accelerator'], dtype='object')


In [62]:
# Inserting data into 03_enroll_date
for i, row in Enroll.iterrows():
    cursor.execute(
        "INSERT IGNORE INTO 03_enroll_date (Email, Accelerator) VALUES (%s, %s)",
        (row['Email'], row['Accelerator'])
    )

In [63]:
# Replacing data with new data into 03_enroll_date
for i, row in Enroll.iterrows():
    cursor.execute(
        "REPLACE INTO 03_enroll_date (Email, Accelerator) VALUES (%s, %s)",
        (row['Email'], row['Accelerator'])
    )

In [64]:
conn.commit()

# Batch Lastlogin Startdate MySQL Table

In [65]:
# Creating new table which for Batch which is taken from incubator graphy sheet
Batch=pd.DataFrame(ce[["Email", 'Last Login','Start Date']])

In [66]:
# Extracting only Enroll_Date
Batch[['Last Login','Time']]=Batch['Last Login'].str.split(' ',expand=True)
Batch=Batch.drop(["Time"],axis=1)

In [67]:
# Function to convert 'Start Date' format
def convert_start_date(date_str):
    if re.match(r'\d{4}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%Y/%m/%d').strftime('%d-%m-%Y')
    return date_str

# Function to convert 'Last Login' format
def convert_last_login(date_str):
    if re.match(r'\d{2}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%d/%m/%y').strftime('%d-%m-%Y')
    return date_str


# Apply the conversion functions to the respective columns
Batch['Start Date'] = Batch['Start Date'].apply(convert_start_date)
Batch['Last Login'] = Batch['Last Login'].apply(convert_last_login)

In [68]:
Batch = Batch.rename(columns={'Start Date': 'Accelerator_Start_Date'})
Batch = Batch.rename(columns={'Last Login': 'Accelerator_Last_Login'})


In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

In [70]:
# Your existing code for inserting data into the database table
for i, row in Batch.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(Batch.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 15_batch_lastlogin_startdate ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in Batch.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [71]:
conn.commit()

# Changing seconds to Percentage

In [72]:
# Saving previous column name 
newcol = list(merged_df.columns)


In [73]:
# Saving previous column name for data1 by removing "Email column"
merged_df_columns = list(merged_df.columns)

# Exclude the 'email' column if it exists
if 'Email' in merged_df_columns:
    merged_df_columns.remove('Email')

merged_df1col = merged_df_columns




In [74]:
# Remove columns starting with "Week" from the list of old columns
oldcol = [col for col in oldcol if not col.startswith("Week")]

# Determine the number of columns to rename (minimum of available columns)
num_columns_to_rename = min(len(merged_df.columns), len(oldcol))

# Rename columns using a dictionary mapping
merged_df.rename(columns=dict(zip(merged_df.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)

# Remove any additional information after the colon in column names
merged_df.columns = merged_df.columns.str.split(':').str[0]

# Remove leading and trailing spaces from column names
merged_df.columns = merged_df.columns.str.strip()


In [75]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')

    # If the first part is 'MM', use the prefix 'MM' and the numbers corresponding to the last two characters
    elif parts[0] == 'MM':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'MM' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [76]:
# Rename the column names using the rename_column function
merged_df.rename(columns=rename_column, inplace=True)

In [77]:
merged_df.columns

Index(['Email', 'SUK1', 'MM01', 'MC15', 'VID38', 'VID39', 'VID40', 'SUK2',
       'VID41', 'VID42', 'VID43', 'VID44', 'MC12', 'VID45', 'VID46', 'MM02',
       'VID47', 'VID48', 'VID21', 'MC13', 'SUK3', 'VID29', 'VID31', 'VID30',
       'MC11', 'VID32', 'VID34', 'VID35', 'VID33', 'SUK4', 'VID36', 'VID37',
       'MC14', 'SUK5'],
      dtype='object')

In [78]:
gd.columns

Index(['Email', 'WK0_SUK_V', 'WK0_MM1', 'WK1_Master1', 'WK2_V1', 'WK2_V2',
       'WK2_V3', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4',
       'WK3_Master2', 'WK4_V1', 'WK4_V2', 'WK4_MM2', 'WK5_V1', 'WK5_V2',
       'WK5_V3', 'WK5_Master3', 'WK6_SUK_V', 'WK7_V1', 'WK7_V2', 'WK7_V3',
       'WK7_Master4', 'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V',
       'WK9_V1', 'WK9_V2', 'WK9_Master5', 'WK10_SUK_V'],
      dtype='object')

In [79]:
# Reading Excel file form our source
excel_file  = pd.read_excel(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\01 Data Format\01 Video Details\Timestamp_of_Videos.xlsx")

In [80]:
# Creating merged_df1 dataframe
merged_df1=pd.DataFrame()

In [81]:
# Iterate over each column in the 'data' DataFrame
for column_name in merged_df.columns:
    # Check if the column exists in the Excel file
    if column_name in excel_file['Code'].values:
        # Get the corresponding value in the "Time" column from the Excel file
        value = excel_file.loc[excel_file['Code'] == column_name, 'Time'].values[0]
        
        # Calculate the percentage value
        percentage_value = (merged_df[column_name] * 100) / value
        
        # Assign the calculated percentage value to a new column in the "merged_df1" DataFrame with the same name as the "Code" column
        merged_df1[column_name] = percentage_value
    else:
        # Handle the case when the column is present in 'merged_df' but not in the Excel file
        print(f"Column '{column_name}' present in 'merged_df' but not in the Excel file. Skipping.")


Column 'Email' present in 'merged_df' but not in the Excel file. Skipping.


In [82]:
# Changing column name back to new column
merged_df.columns = newcol 
merged_df1.columns = merged_df1col

In [83]:
# Creating a function "Max_Value" for converting % max 100 if values are above 100
def Max_Value(value):
    if value >=100:
        return 100
    else:
        return value

In [84]:
# Applying Max_Value function
per1=merged_df1.iloc[:,0:]
per1c=per1.columns
merged_df1[per1c]=merged_df1[per1c].applymap(Max_Value)

C:\Users\spjay\AppData\Local\Temp\ipykernel_54980\3614295319.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df1[per1c]=merged_df1[per1c].applymap(Max_Value)


# Code for converting seconds to hours

In [85]:
# Creating a function for converting seconds to hours
def Convert_Hours(seconds):
    hours = seconds / (3600)
    return hours
n = 5400
print(Convert_Hours(n))


1.5


In [86]:
# Applying Convert_Hours function
col1 = merged_df.iloc[:,1:]
col=col1.columns
merged_df[col]=merged_df[col].apply(Convert_Hours)

# Code for Recorded Videos: Total hours & Percentage

In [87]:
# Define the week number up to which you want to include columns
end_week = 14

In [88]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Total = merged_df[cols_to_select]

In [89]:
merged_df["Pre_Recorded_Total_Hours"]=Recorded_Total.sum(axis=1)

In [90]:
# Create a list of all column names up to the end week
cols_to_select_per = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in merged_df1.columns:
            cols_to_select_per.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Percent = merged_df1[cols_to_select_per]

#### As below, we have given all values as 0. Because of that, we are calculating the average for all the week video. If there is NA, then it is not getting counted in average, such as suk and masterclass. The average will be just masterclass and suk happen.

In [91]:
# Filling null values with 0
Recorded_Percent.fillna(0, inplace=True)

C:\Users\spjay\AppData\Local\Temp\ipykernel_54980\3319348735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Recorded_Percent.fillna(0, inplace=True)


In [92]:
merged_df["Pre_Recorded_Percentage"]=Recorded_Percent.mean(axis=1)

# Code for SUK Recorded Videos: Total hours & Percentage

In [93]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in merged_df.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Total = merged_df[cols_to_select]

In [94]:
merged_df["SUK_Total_Hours"]=SUK_Recorded_Total.sum(axis=1)

In [95]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in merged_df1.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Percent = merged_df1[cols_to_select]

In [97]:
merged_df["SUK_Percentage"]=SUK_Recorded_Percent.mean(axis=1)

# Code for Master Class: Total hours & Percentage

In [98]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 2):
    for masterclass in range(1, 50):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Total = merged_df[cols_to_select]


In [99]:
merged_df["Masterclass_Total_Hours"]=Masterclass_Total.sum(axis=1)

In [100]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for masterclass in range(1, 10):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in merged_df1.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Percent = merged_df1[cols_to_select]


In [101]:
merged_df["Masterclass_Percentage"]=Masterclass_Percent.mean(axis=1)

# Code for MM Class: Total hours & Percentage

In [102]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_MM{video}'
        if col_name in merged_df.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
MM_Total = merged_df[cols_to_select]

In [103]:
merged_df["MM_Total_Hours"]=MM_Total.sum(axis=1)

In [104]:
# Create a list of all column names up to the end week
cols_to_select_per = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_MM{video}'
        if col_name in merged_df1.columns:
            cols_to_select_per.append(col_name)

# Select the desired columns and create a new DataFrame
MM_Percent = merged_df1[cols_to_select_per]

In [105]:
merged_df["MM_Percentage"]=MM_Percent.mean(axis=1)

# Code for Course: Total hours & Percentage

In [106]:
# select columns from data where the column names start with "WK"
selected_cols = [col for col in merged_df.columns if col.startswith('WK')]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Total = merged_df[selected_cols]

In [107]:
merged_df["Course_Total_Hours"]=Whole_Program_Total.sum(axis=1)

In [108]:
merged_df.columns

Index(['Email', 'WK0_SUK_V', 'WK0_MM1', 'WK1_Master1', 'WK2_V1', 'WK2_V2',
       'WK2_V3', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4',
       'WK3_Master2', 'WK4_V1', 'WK4_V2', 'WK4_MM2', 'WK5_V1', 'WK5_V2',
       'WK5_V3', 'WK5_Master3', 'WK6_SUK_V', 'WK7_V1', 'WK7_V2', 'WK7_V3',
       'WK7_Master4', 'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V',
       'WK9_V1', 'WK9_V2', 'WK9_Master5', 'WK10_SUK_V',
       'Pre_Recorded_Total_Hours', 'Pre_Recorded_Percentage',
       'SUK_Total_Hours', 'SUK_Percentage', 'Masterclass_Total_Hours',
       'Masterclass_Percentage', 'MM_Total_Hours', 'MM_Percentage',
       'Course_Total_Hours'],
      dtype='object')

##### Here we are taking all Suk, Masterclass,mm and recording for course percantage.

In [109]:
# select columns from gd where the column names start with "WK"
selected_cols_1 = [col for col in merged_df.columns if col.startswith("Pre_Recorded_Percentage") or col.startswith("SUK_Percentage") or col.startswith("Masterclass_Percentage") or col.startswith("MM_Percentage")or col.startswith("Mentor_Interaction_Percentage")]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Percent = merged_df[selected_cols_1]

In [110]:
Whole_Program_Percent.head()

,Pre_Recorded_Percentage,SUK_Percentage,Masterclass_Percentage,MM_Percentage
0,1.336675,40.166667,21.194444,0.250000
1,97.969399,69.244444,80.600000,79.694444
2,96.669012,100.000000,81.394444,55.347222
3,33.333333,60.000000,60.000000,0.000000
4,0.000000,20.761111,20.000000,0.000000


In [111]:
Whole_Program_Percent# Filling null values with 0
Whole_Program_Percent.fillna(0, inplace=True)

C:\Users\spjay\AppData\Local\Temp\ipykernel_54980\1585490648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Whole_Program_Percent.fillna(0, inplace=True)


In [112]:
merged_df["Course_Percentage"]=Whole_Program_Percent.mean(axis=1)

In [113]:
merged_df=merged_df.round(2)

In [114]:
# Filling null values with 0
merged_df.fillna(0, inplace=True)


In [115]:
# Copy merged_df to merged_d
merged_db = merged_df.copy()

# Merging the file with genral sheet

In [116]:
# Renaming the column names
gi.rename(columns={'Email ID' : 'Email'}, inplace=True)

In [117]:
# Drop columns starting with 'WK'
gi = gi.drop(['Accelerator Batch'], axis=1)

In [119]:
# Merge with gi
merged_df = pd.merge(gi, merged_df, on='Email', how='outer')


In [120]:
# Function to rearrange columns
def rearrange_columns(merged_df):
    def extract_week_number(col):
        match = re.search(r'WK(\d+)', col)
        return int(match.group(1)) if match else -1

    wk_columns = sorted([col for col in merged_df.columns if col.startswith('WK')], key=extract_week_number)
    total_columns = [col for col in merged_df.columns if col.endswith('Hours')]
    percentage_columns = [col for col in merged_df.columns if col.endswith('Percentage')]
    other_columns = [col for col in merged_df.columns if col not in wk_columns + total_columns + percentage_columns]
    
    new_column_order = other_columns + wk_columns + total_columns + percentage_columns
    return merged_df[new_column_order]

# Applying the function
merged_df = rearrange_columns(merged_df)

In [121]:
# Rounding off the values
merged_df=merged_df.round(2)

In [122]:
# merged_df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Accelerator Course Attendance.csv')
merged_df.to_csv(output_path, index=False)


# Merging the file with Mentor Interaction sheet

In [123]:
# importing dataframe 
men = pd.read_csv(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\output\Mentor Interaction Attendance.csv")

In [124]:
# Rename columns function
def rename_columns(men):
    new_columns = {}
    for col in men.columns:
        if 'Hours Spent - Week' in col:
            week_num = col.split(' - Week ')[1]
            new_col_name = f'WK{week_num}_Mentor_Interaction'
            new_columns[col] = new_col_name
    men.rename(columns=new_columns, inplace=True)

# Applying the function
rename_columns(men)

In [125]:
men.columns

Index(['Mentee ID', 'Email ID', 'Mentee Name', 'Mobile Number', 'Group Name',
       'Total Sessions - Week 1', 'Average Duration - Week 1',
       'Attended Sessions - Week 1', 'WK1_Mentor_Interaction',
       '% Attendance - Week 1', 'Total Sessions - Week 2',
       'Average Duration - Week 2', 'Attended Sessions - Week 2',
       'WK2_Mentor_Interaction', '% Attendance - Week 2',
       'Total Sessions - Week 3', 'Average Duration - Week 3',
       'Attended Sessions - Week 3', 'WK3_Mentor_Interaction',
       '% Attendance - Week 3', 'Total Sessions - Week 4',
       'Average Duration - Week 4', 'Attended Sessions - Week 4',
       'WK4_Mentor_Interaction', '% Attendance - Week 4',
       'Total Sessions - Week 5', 'Average Duration - Week 5',
       'Attended Sessions - Week 5', 'WK5_Mentor_Interaction',
       '% Attendance - Week 5', 'Total Sessions - Week 6',
       'Average Duration - Week 6', 'Attended Sessions - Week 6',
       'WK6_Mentor_Interaction', '% Attendance - Week

In [126]:
# Remove unwanted Sign from column names
men = men.rename(columns=lambda x: x.replace('Hours Spent Overall', 'Mentor_Interaction_Total_Hours').replace('% Attendance Overall', 'Mentor_Interaction_Percentage'))


In [127]:
# Get the columns to keep
columns_to_keep = [col for col in men.columns if col.startswith("Mentee ID") or col.startswith("WK") or col.startswith("Mentor_Interaction_Total_Hours")or col.startswith("Mentor_Interaction_Percentage")]

# Drop the columns not in columns_to_keep
men.drop(columns=[col for col in men.columns if col not in columns_to_keep], inplace=True)


In [128]:
# Merge with merged_df
overall = pd.merge(merged_df, men, on='Mentee ID', how='outer')


# Code for Program: Total hours & Percentage

In [129]:
# select columns from data where the column names start with "WK"
selected_cols = [col for col in overall.columns if col.startswith('WK')]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Total = overall[selected_cols]

In [130]:
overall["Program_Total_Hours"]=Whole_Program_Total.sum(axis=1)

In [131]:
# select columns from data where the column names start
selected_cols_1 = [col for col in overall.columns if col.startswith("Pre_Recorded_Percentage") or col.startswith("SUK_Percentage") or col.startswith("Masterclass_Percentage") or col.startswith("MM_Percentage")or col.startswith("Mentor_Interaction_Percentage")]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Percentage = overall[selected_cols_1]

In [132]:
Whole_Program_Percentage.head()

,Pre_Recorded_Percentage,SUK_Percentage,Masterclass_Percentage,MM_Percentage,Mentor_Interaction_Percentage
0,0.31,0.08,0.05,0.00,54.55
1,78.73,22.59,20.23,28.90,50.00
2,0.00,0.01,34.46,3.69,27.27
3,93.48,52.00,54.53,44.47,13.64
4,0.00,0.00,0.00,0.00,0.00


In [133]:
overall["Program_Percentage"]=Whole_Program_Percentage.mean(axis=1)

In [134]:
overall.columns

Index(['Mentee ID', 'Email', 'Mentee Name', 'Mobile Number', 'Group Name',
       'WK0_SUK_V', 'WK0_MM1', 'WK1_Master1', 'WK2_V1', 'WK2_V2', 'WK2_V3',
       'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master2',
       'WK4_V1', 'WK4_V2', 'WK4_MM2', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_Master3', 'WK6_SUK_V', 'WK7_V1', 'WK7_V2', 'WK7_V3', 'WK7_Master4',
       'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V', 'WK9_V1', 'WK9_V2',
       'WK9_Master5', 'WK10_SUK_V', 'Pre_Recorded_Total_Hours',
       'SUK_Total_Hours', 'Masterclass_Total_Hours', 'MM_Total_Hours',
       'Course_Total_Hours', 'Pre_Recorded_Percentage', 'SUK_Percentage',
       'Masterclass_Percentage', 'MM_Percentage', 'Course_Percentage',
       'WK1_Mentor_Interaction', 'WK2_Mentor_Interaction',
       'WK3_Mentor_Interaction', 'WK4_Mentor_Interaction',
       'WK5_Mentor_Interaction', 'WK6_Mentor_Interaction',
       'WK7_Mentor_Interaction', 'WK8_Mentor_Interaction',
       'WK9_Mentor_Interaction

In [135]:
# Function to rearrange columns
def rearrange_columns(overall):
    def extract_week_number(col):
        match = re.search(r'WK(\d+)', col)
        return int(match.group(1)) if match else -1

    wk_columns = sorted([col for col in overall.columns if col.startswith('WK')], key=extract_week_number)
    total_columns = [col for col in overall.columns if col.endswith('Hours')]
    percentage_columns = [col for col in overall.columns if col.endswith('Percentage')]
    other_columns = [col for col in overall.columns if col not in wk_columns + total_columns + percentage_columns]
    
    new_column_order = other_columns + wk_columns + total_columns + percentage_columns
    return overall[new_column_order]

# Applying the function
overall = rearrange_columns(overall)

In [136]:
# Filling null values with 0
overall.fillna(0, inplace=True)

In [137]:
overall.columns

Index(['Mentee ID', 'Email', 'Mentee Name', 'Mobile Number', 'Group Name',
       'WK0_SUK_V', 'WK0_MM1', 'WK1_Master1', 'WK1_Mentor_Interaction',
       'WK2_V1', 'WK2_V2', 'WK2_V3', 'WK2_SUK_V', 'WK2_Mentor_Interaction',
       'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master2',
       'WK3_Mentor_Interaction', 'WK4_V1', 'WK4_V2', 'WK4_MM2',
       'WK4_Mentor_Interaction', 'WK5_V1', 'WK5_V2', 'WK5_V3', 'WK5_Master3',
       'WK5_Mentor_Interaction', 'WK6_SUK_V', 'WK6_Mentor_Interaction',
       'WK7_V1', 'WK7_V2', 'WK7_V3', 'WK7_Master4', 'WK7_Mentor_Interaction',
       'WK8_V1', 'WK8_V2', 'WK8_V3', 'WK8_V4', 'WK8_SUK_V',
       'WK8_Mentor_Interaction', 'WK9_V1', 'WK9_V2', 'WK9_Master5',
       'WK9_Mentor_Interaction', 'WK10_SUK_V', 'WK10_Mentor_Interaction',
       'WK11_Mentor_Interaction', 'WK12_Mentor_Interaction',
       'WK13_Mentor_Interaction', 'Pre_Recorded_Total_Hours',
       'SUK_Total_Hours', 'Masterclass_Total_Hours', 'MM_Total_Hours',
       'Course_Total_Hours'

In [138]:
overall.head()

,Mentee ID,Email,Mentee Name,Mobile Number,Group Name,WK0_SUK_V,WK0_MM1,WK1_Master1,WK1_Mentor_Interaction,WK2_V1,WK2_V2,WK2_V3,WK2_SUK_V,WK2_Mentor_Interaction,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK3_Master2,WK3_Mentor_Interaction,WK4_V1,WK4_V2,WK4_MM2,WK4_Mentor_Interaction,WK5_V1,WK5_V2,WK5_V3,WK5_Master3,WK5_Mentor_Interaction,WK6_SUK_V,WK6_Mentor_Interaction,WK7_V1,WK7_V2,WK7_V3,WK7_Master4,WK7_Mentor_Interaction,WK8_V1,WK8_V2,WK8_V3,WK8_V4,WK8_SUK_V,WK8_Mentor_Interaction,WK9_V1,WK9_V2,WK9_Master5,WK9_Mentor_Interaction,WK10_SUK_V,WK10_Mentor_Interaction,WK11_Mentor_Interaction,WK12_Mentor_Interaction,WK13_Mentor_Interaction,Pre_Recorded_Total_Hours,SUK_Total_Hours,Masterclass_Total_Hours,MM_Total_Hours,Course_Total_Hours,Mentor_Interaction_Total_Hours,Program_Total_Hours,Pre_Recorded_Percentage,SUK_Percentage,Masterclass_Percentage,MM_Percentage,Course_Percentage,Mentor_Interaction_Percentage,Program_Percentage
0,KA03_01,rangarirohinireddy@gmail.com,Rohini Reddy Rangari,6.281661e+09,BIO_01Neha,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.5,0.00,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,1.0,0.00,2.0,0.0,1.5,0.0,0.00,0.00,0.00,0.00,0.01,8.0,8.00,0.31,0.08,0.05,0.00,0.11,54.55,10.998
1,KA03_02,namrtad82@gmail.com,Namrta Dixit,9.630908e+09,BIO_01Neha,1.50,0.54,1.11,1.0,0.03,0.08,0.03,0.13,1.5,0.06,0.08,0.09,0.11,0.00,0.0,0.05,0.03,0.04,1.0,0.03,0.03,0.00,0.00,1.0,0.00,0.0,0.05,0.08,0.00,0.00,0.0,0.05,0.03,0.04,0.04,0.00,1.0,0.04,0.06,0.0,0.0,0.00,1.0,0.0,0.0,0.0,1.03,1.63,1.12,0.58,4.36,6.5,10.83,78.73,22.59,20.23,28.90,37.62,50.00,40.090
2,KA03_03,amalendus4nair@gmail.com,Amalendu Sasikumar Nair,9.496757e+09,BIO_01Neha,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.5,0.00,0.00,0.00,0.00,1.00,1.0,0.00,0.00,0.04,0.0,0.00,0.00,0.00,0.72,1.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,1.72,0.04,1.76,3.5,5.26,0.00,0.01,34.46,3.69,9.54,27.27,13.086
3,KA03_26,anaswara2928@gmail.com,Anaswara H,7.736603e+09,BIO_01Neha,1.39,0.69,0.29,0.0,0.09,0.10,0.04,1.07,0.0,0.35,0.04,0.08,0.17,0.24,0.0,0.04,0.05,0.20,0.0,0.04,0.05,0.17,1.16,1.0,0.34,0.0,0.08,0.09,0.08,1.04,0.0,0.03,0.04,0.05,0.09,0.14,0.0,0.04,0.04,0.2,0.0,0.12,1.0,0.0,0.0,0.0,1.76,3.06,2.93,0.89,8.64,2.0,10.64,93.48,52.00,54.53,44.47,61.12,13.64,51.624
4,KA03_05,swathi.deepak@mbty.christuniversity.in,Swathi Nayak,9.606190e+09,BIO_01Neha,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000


In [139]:
# Drop columns starting with 'WK'
overall = overall.loc[:, ~overall.columns.str.startswith('WK')]

In [140]:
# Drop columns starting with 'WK'
overall = overall.loc[:, ~overall.columns.str.startswith('Course')]

In [141]:
# Rounding off
overall=overall.round(2)

In [142]:
# merged_df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\05 Accelerator\Accelerator 3.0\output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Overall Attendance Tracking.csv')
overall.to_csv(output_path, index=False)


In [143]:
overall.columns

Index(['Mentee ID', 'Email', 'Mentee Name', 'Mobile Number', 'Group Name',
       'Pre_Recorded_Total_Hours', 'SUK_Total_Hours',
       'Masterclass_Total_Hours', 'MM_Total_Hours',
       'Mentor_Interaction_Total_Hours', 'Program_Total_Hours',
       'Pre_Recorded_Percentage', 'SUK_Percentage', 'Masterclass_Percentage',
       'MM_Percentage', 'Mentor_Interaction_Percentage', 'Program_Percentage'],
      dtype='object')

In [144]:
# List of columns to keep
columns_to_keep = ['Email',
    'Mentor_Interaction_Total_Hours',
    'Pre_Recorded_Total_Hours',
    'SUK_Total_Hours',
    'Masterclass_Total_Hours',
    'MM_Total_Hours',
    'Program_Total_Hours',
    'Pre_Recorded_Percentage',
    'SUK_Percentage',
    'Masterclass_Percentage',
    'MM_Percentage',
    'Mentor_Interaction_Percentage',
    'Program_Percentage'
]

# Create a new DataFrame with the filtered columns
overall_db = overall[columns_to_keep]

# Attendance monitoring & Overall MySQL table

In [145]:
oldcol = [col for col in oldcol if not col.startswith("Week")]

In [146]:
oldcol

['Email',
 'SUK ! : Kalpana Accelerator Mentees Orientation Session - 664dbaa7bb3f0b60dc216e98',
 'MM ! : Mentor - mentee mixer event - 664dbb23583bd42868f1b6ca',
 'Master Class !% : Cutting-edge AI tools to find credible research information - 664dbedc1723ba4c83b6b5e8',
 'Video #* : What is creativity & growth mindset? - 62b3fe500cf2f5120a60c95b',
 'Video #? : Why is it important to have a creativity & growth mindset ? - 62b3fe750cf2f5120a60c972',
 'Video $+ : How to develop a creative growth mindset ? - 62b3fe6d0cf2f5120a60c971',
 'SUK @ : Session 1 - Saturday, 29 June 2024 - 664dc110583bd42868f1b87f',
 'Video $! : What is critical thinking and why is it important ? - 62ac23340cf29ec26dfc3e91',
 'Video $@ : The scientific temperament - 62b3fec80cf2f5120a60c981',
 'Video $# : Scientific temper & critical thinking - 62b3fec50cf2f5120a60c97e',
 'Video $$ : The scientific method - 62b3feee0cf2f5120a60c99d',
 'Master Class !@ : STEM habits and Scientific Temperament - Saturday, 6 July 202

In [147]:
num_columns_to_rename = min(len(merged_db.columns), len(oldcol))
print(num_columns_to_rename)

34


In [148]:
merged_db.rename(columns=dict(zip(merged_db.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)

In [149]:
merged_db.columns = merged_db.columns.str.strip()

In [150]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')

    # If the first part is 'MM', use the prefix 'MM' and the numbers corresponding to the last two characters
    elif parts[0] == 'MM':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'MM' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [151]:
# Rename the column names using the rename_column function
merged_db.rename(columns=rename_column, inplace=True)

In [153]:
# Rename the column name
merged_db = merged_db.rename(columns={'Course_Total_Hours': 'Central_Curriclum_Total_Hours'})
merged_db = merged_db.rename(columns={'Course_Percentage': 'Central_Curriclum_Percentage'})


In [ ]:
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor() 

In [155]:
# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM 19_accelerator_and_attendance_monitoring")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Pre_Recorded_Total_Hours'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in merged_db.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "float" 
            alter_query = f"ALTER TABLE 19_accelerator_and_attendance_monitoring ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)

In [156]:
# Your existing code for inserting data into the database table
for i, row in merged_db.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(merged_db.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 19_accelerator_and_attendance_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in merged_db.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [157]:
# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM 20_accelerator_overall_attendance_tracking")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Pre_Recorded_Percentage'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in overall_db.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "float" 
            alter_query = f"ALTER TABLE 20_accelerator_overall_attendance_tracking ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)

In [158]:
# Your existing code for inserting data into the database table
for i, row in overall_db.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(overall_db.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 20_accelerator_overall_attendance_tracking ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in overall_db.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [159]:
conn.commit()